# Выручка на пользователя, заказы на платящего, средний чек

*В рамках байесовского подхода сравниваются выручка на пользователя, заказы на платящего, средний чек.*

* [Введение](#Введение)  
* [Выручка на пользователя](#Выручка-на-пользователя)
* [Заказы на платящего](#Заказы-на-платящего)
* [Средний чек](#Средний-чек)
* [Заключение](#Заключение) 

# Введение

# Выручка на пользователя

В А/Б тестах как итоговая величина обычно интересна выручка на пользователя, попавшего в тест.  
Относительная разность этих величин между группами позволяет оценить эффект при увеличении трафика в группы.  

Для моделирования выручку на пользователя удобно представить в виде конверсии в оплату и выручки на платящего.  

$$
\mbox{выручка на пользователя} = \mbox{конверсия в оплату} \cdot \mbox{выручка на платящего}
\\
P(x) = p_{не платили} P(x=0) + p_{оплатили} P_{выручка на платящего}(x)
$$

Оценка конверсии в оплату делалась ранее.  
Выручка на платящего в удобно моделировать лог-нормальное распределение и распределение Парето.  
В транзакционных сервисах вроде маркетплейсов выручка на платяещего хорошо моделируется лог-нормальным распределением.

Для моделирования выручки на пользователя могут быть применимы распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution)] и лог-нормальное распределение [[LognormDist](https://en.wikipedia.org/wiki/Log-normal_distribution)].  

https://arxiv.org/abs/cond-mat/0412004  
https://academic.oup.com/bioscience/article/51/5/341/243981?login=false  
https://en.wikipedia.org/wiki/Gibrat%27s_law


В модельных примерах Парето появляется если есть ограничение метрики снизу, логнормальное - если может быть нулем [Mitz]. Похожий критерий может быть применим к выручке на пользователя: Парето если есть минимальная сумма, логнормальное если может быть 0.


Лог-нормальное распределение [[LognormDist](https://en.wikipedia.org/wiki/Log-normal_distribution),
[SciPyLognorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html)]

$$
\begin{split}
X & \sim Lognormal(\mu, \sigma^2)
\\
\ln(X) & \sim Norm(\mu, \sigma^2)
\\
f(x) & = \frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
= \frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln x / e^{\mu})^2}{2 \sigma^2}}
\\
E[x] & = e^{\left(\mu + \sigma^2/2\right)}
\end{split}
$$

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

from collections import namedtuple

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for sigma, loc, scale in [(1, 0, 1), (2, 0, 1), (1, 0, 2)]:
    fig.add_trace(go.Scatter(x=x, y=stats.lognorm.pdf(x, s=sigma, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f's={sigma}, loc={loc}, scale={scale}'))
fig.update_layout(title='Lognormal',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}
$$

Сопряженное априорное распределение - произведение нормального и гамма-распределений аналогично сопряженному распределению для нормального распределения [ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior), [compendum of priors], [GammaDist](https://en.wikipedia.org/wiki/Gamma_distribution), [SciPyStatsGamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html)

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \mu, \sigma) = 
\frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
$$

$$
P(model) = 
Norm(\mu | \tau; \mu_0, k_0) Gamma(\tau; a, b) = 
(\tau k_0/ \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
\prod_i^N \left( \frac{\tau}{x_i \pi} \right)^{1/2} e^{- \tau (\ln(x_i)-\mu)^2}
\left( \frac{\tau k_0}{\pi} \right)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& = Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\end{split}
$$

Параметры
$$
\begin{split}
a_N & = a + N/2
\\
k_N & = k_0 + N
\\
\mu_N & = \frac{k_0 \mu_0 + \sum_i^N \ln x_i}{k_0 + N}
\\
b_N & = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N (\ln x_i )^2
\end{split}
$$

Исходные значения параметров:

$$
\mu_0 = ,
\\ 
k_0 = 1,
\\
a_0 = , b_0 = .
$$

Маржинальные распределения

По $\tau$ гамма-распределение

$$
P(\tau | data) = Gamma(\tau; a_N, b_N)
$$

По $\mu$ [смещенное $t$-распределение](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Location-scale_t_distribution)  

$$
P(\mu | data) = lst \left( \mu; \nu = 2 a_N, \mu_t = \mu_N, \tau= \left( \frac{b_N}{2 a_N k_N} \right)^{1/2} \right)
$$

In [ ]:
ConjugateLogNormalParams = namedtuple('ConjugateLogNormalParams', 'mu k a b')

def initial_params_lognormal(mu, k=1, a=2, b=1):
    return ConjugateLogNormalParams(mu=mu, k=k, a=a, b=b)

def posterior_params_lognormal(data, initial_pars):
    N = len(data)
    lnx = np.array([np.log(x) for x in data])
    a_n = initial_pars.a + N / 2
    k_n = initial_pars.k + N
    mu_n = (initial_pars.k * initial_pars.mu + np.sum(lnx)) / (initial_pars.k + N)    
    b_n = initial_pars.b + initial_pars.k * initial_pars.mu**2 - k_n * mu_n**2 + np.sum(lnx*lnx)
    return ConjugateLogNormalParams(mu=mu_n, k=k_n, a=a_n, b=b_n)

def posterior_rvs(params, nsamp):
    tau = stats.gamma.rvs(a=params.a, scale=1/params.b, size=nsamp)
    sigma_m = 1 / np.sqrt(2 * params.k * tau)
    mu = stats.norm.rvs(loc=params.mu, scale=sigma_m, size=nsamp)
    sigma_x = 1 / np.sqrt(2 * tau)
    x = stats.lognorm.rvs(s=sigma_x, loc=0, scale=np.exp(mu), size=nsamp)
    return x, mu, tau

def posterior_marginal_mu_dist(params):
    nu = 2 * params.a
    mu_t = params.mu
    tau = np.sqrt(params.b/(2.0 * params.a * params.k))
    return stats.t(df=nu, loc=mu_t, scale=tau)

def posterior_marginal_tau_dist(params):
    return stats.gamma(a=params.a, scale=1/params.b)


s = 1.3
loc = 0
scale = 0.5
nsample = 10000

exact_dist = stats.lognorm(s=s, loc=loc, scale=scale)

data = exact_dist.rvs(nsample)

pars = initial_params_lognormal(mu=1, k=1, a=2, b=1)
pars = posterior_params_lognormal(data, pars)

post_samp, post_samp_mu, post_samp_tau = posterior_rvs(pars, nsamp=200000)
postdist_mu = posterior_marginal_mu_dist(pars)
postdist_tau = posterior_marginal_tau_dist(pars)

x = np.linspace(0, 20, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), name='Lognorm'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Post Samp Hist', nbinsx=10000))
fig.update_layout(title='Lognormal Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  xaxis_range=[-1,30],
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(-5, 20, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=np.log(scale), scale=s), name='Norm'))
fig.add_trace(go.Histogram(x=np.log(data), histnorm='probability density', name='ln(data)', nbinsx=1000))
fig.add_trace(go.Histogram(x=np.log(post_samp), histnorm='probability density', name='ln(post)', 
                           opacity=0.5, nbinsx=1000))
fig.update_layout(title='ln(x)',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[-1,30],
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(-1, 3, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_mu.pdf(x), mode='lines', name=f'Posterior Mu'))
fig.add_vline(np.log(scale), name='ln(scale)')
fig.update_layout(title='Mu Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

x = np.linspace(-1, 3, 2000)
fig = go.Figure()
fig.add_vline(1/(2*s*s), name='tau=1/(2 s^2)')
fig.add_trace(go.Scatter(x=x, y=postdist_tau.pdf(x), mode='lines', name=f'Posterior Tau'))
fig.update_layout(title='Tau Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

nsamp = 50000
post_samp_sigma = 1 / np.sqrt(2 * post_samp_tau)
post_samp_mean = np.exp(post_samp_mu + post_samp_sigma**2 / 2)
x = np.linspace(0, 3, 2000)
fig = go.Figure()
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=post_samp_mean, histnorm='probability density', name='Post Samp Mean', nbinsx=10000))
fig.update_layout(title='Mean Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Проще оценивать нормальное распределение логарифма

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu k a b')

def initial_params_normal(mu, k=1, a=2, b=1):
    return ConjugateNormalParams(mu=mu, k=k, a=a, b=b)

def posterior_params_normal(data, initial_pars):
    N = len(data)
    a_n = initial_pars.a + N / 2
    k_n = initial_pars.k + N
    mu_n = (initial_pars.k * initial_pars.mu + np.sum(data)) / (initial_pars.k + N)    
    b_n = initial_pars.b + initial_pars.k * initial_pars.mu**2 - k_n * mu_n**2 + np.sum(data*data)
    return ConjugateNormalParams(mu=mu_n, k=k_n, a=a_n, b=b_n)

def posterior_normal_rvs(params, nsamp):
    tau = stats.gamma.rvs(a=params.a, scale=1/params.b, size=nsamp)
    sigma_m = 1 / np.sqrt(2 * params.k * tau)
    mu = stats.norm.rvs(loc=params.mu, scale=sigma_m, size=nsamp)
    sigma_x = 1 / np.sqrt(2 * tau)
    x = stats.norm.rvs(loc=mu, scale=sigma_x, size=nsamp)
    return x

def posterior_normal_marginal_mu_dist(params):
    nu = 2 * params.a
    mu_t = params.mu
    tau = np.sqrt(params.b/(2.0 * params.a * params.k))
    return stats.t(df=nu, loc=mu_t, scale=tau)

def posterior_normal_marginal_tau_dist(params):
    return stats.gamma(a=params.a, scale=1/params.b)


s = 1.3
loc = 0
scale = 0.5
nsample = 10000

exact_dist = stats.lognorm(s=s, loc=loc, scale=scale)
data = exact_dist.rvs(nsample)

log_exact_dist = stats.norm(loc=np.log(scale), scale=s)
log_data = np.log(data)

pars = initial_params_normal(mu=1, k=1, a=2, b=1)
pars = posterior_params_normal(log_data, pars)

log_post_samp = posterior_normal_rvs(pars, nsamp=100000)
postdist_mu = posterior_normal_marginal_mu_dist(pars)
postdist_tau = posterior_normal_marginal_tau_dist(pars)

x = np.linspace(-20, 20, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=log_exact_dist.pdf(x), name='Exact ln(x)'))
fig.add_trace(go.Histogram(x=log_data, histnorm='probability density', name='ln(Samp)', nbinsx=1000))
fig.add_trace(go.Histogram(x=log_post_samp, histnorm='probability density', name='Post Samp', opacity=0.5, nbinsx=1000))
fig.update_layout(title='Ln(data) Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(-20, 20, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), name='Exact'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Samp', nbinsx=1000))
fig.add_trace(go.Histogram(x=np.exp(log_post_samp), histnorm='probability density', name='Post Samp', opacity=0.5, nbinsx=10000))
fig.update_layout(title='Lognormal Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()


x = np.linspace(-1, 10, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_mu.pdf(x), mode='lines', name=f'Posterior Mu'))
fig.add_vline(log_exact_dist.mean(), name='Ln(x) Mean')
fig.update_layout(title='Mu Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()

x = np.linspace(-3, 3, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_tau.pdf(x), mode='lines', name=f'Posterior Tau'))
fig.add_vline(1/(2 * log_exact_dist.std()**2), name='Original Distribution Sigma')
fig.update_layout(title='Tau Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

# Заказы на платящего

Для заказов на пользователя удобнее рассматривать распределения с дискретными значениями.  

$$
P(n), \quad n \in 1, 2, \dots
$$

Возможные варианты моделей - разность накопленных функций распределений логнормального или Парето:  

$$
P(n) = CDF_{Lognorm(\mu, \sigma)}(n+1) - CDF_{Lognorm(\mu, \sigma)}(n)
$$

$$
P(n) = CDF_{Pareto(s)}(n+1) - CDF_{Pareto(s)}(n)
$$

Парето:

$$
CDF_{P(x_m, c)}(x) = 
\begin{cases}
1 - \left(\dfrac{x_m}{x}\right)^{c} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

Для заказов $x_m = 1$.

$$
P(n) = 
\dfrac{1}{n^c} - \dfrac{1}{(n+1)^{c}}
$$

Для логнормального CDF выражается через функцию ошибок https://en.wikipedia.org/wiki/Log-normal_distribution#Cumulative_distribution_function .  
Не ясно, получится ли посчитать сопряженное.

Еще вариант - распределение Ципфа https://en.wikipedia.org/wiki/Zipf%27s_law#Formal_definition .

$$
f(k; N, s) = \frac{1}{H_{N,s}} \frac{1}{k^s},
\quad
H_{N,s} = \sum_{k=1}^N \frac{1}{k^s}
$$

Еще вариант - свои вероятности под каждое количество заказов $p_n$.
С учетом перестановок для нескольких пользователей мультиномиальное распределение https://en.wikipedia.org/wiki/Multinomial_distribution .

$$
P(n_1, \dots, n_N) = \frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}},
$$

где $n_i$ - количество пользователей с $i$ заказами, $p_i$ - вероятность сделать $i$ заказов.   
Нужно определиться с максимальным количеством заказов $N$. 

Распределение Дирихле сопряженное к нему https://en.wikipedia.org/wiki/Dirichlet_distribution .


*todo: невозрастающие $1 \ge p_1 \ge p_2 \ge \dots \ge p_N$ .*

In [ ]:
def pareto_d(n, c):
    return 1/n**c - 1/(n+1)**c

def zipfs(n, s, N):
    h_Ns = np.sum([1/i**s for i in range(1, N+1)])
    return 1/h_Ns * 1/n**s

x = np.arange(1, 20)
fig = go.Figure()
for s in [(1), (1.1), (2)]:
    fig.add_trace(go.Scatter(x=x, y=pareto_d(n=x, c=s), 
                             mode='lines+markers', 
                             name=f'Pareto, s={s}'))
    fig.add_trace(go.Scatter(x=x, y=zipfs(n=x, s=s, N=30), 
                             mode='lines+markers', 
                             name=f'Zipf, s={s}'))
fig.update_layout(title='Pareto, Zipf',
                  xaxis_title='x',
                  yaxis_title='Prob',
                  hovermode="x",
                  height=550)
fig.show()

Сопряженное распределение для мультиномиального распределения

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = \frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}}
$$

$$
P(model) = 
Dir \left( p_{1}, \dots, p_{N}; \alpha_{1}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{1}, \dots, \alpha_{N} )} \prod_{i=1}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=1}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{1}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=1}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=1}^{N} \alpha_{i} \right)}
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{(n_1 + \dots + n_N)!}{n_{1}! \dots n_{N}!} p_{1}^{n_{1}} \dots p_{N}^{n_{N}}
\dfrac{1}{B(\alpha_{1}, \dots ,\alpha_{N})} \prod _{i=1}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=1}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{1}, \dots, p_{N}; n_1 + \alpha_{1}, \dots, n_N + \alpha_{N} \right)
\end{split}
$$

Маржинальные распределения - бета-распределения
$$
\begin{split}
f(p_i) = 
Beta( p_i; \alpha_i, \alpha_0 - \alpha_i ),
\quad
\alpha_0 = \sum_{i=1}^{N} \alpha_i
\end{split}
$$
Биномиальное распределение по $i$-му исходу и всем остальным исходам. 

Среднее:

P(среднее A > среднее B):

In [ ]:
def initial_params_dr(N):
    return np.ones(N)

def posterior_params_dr(data, initial_pars):
    u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in zip(u, c):
        post_pars[k - 1] = post_pars[k - 1] + v
    return post_pars

def posterior_nords_rvs(params, nsamp):
    nords = np.empty(nsamp)
    probs = stats.dirichlet.rvs(alpha=params, size=nsamp)
    for i, p in enumerate(probs):
        nords[i] = np.argmax(stats.multinomial.rvs(n=1, p=p)) + 1
    return nords

def posterior_marginal_dist_pi(i, params):
    imin = 1
    p_i = stats.beta(a=params[i - imin], b=np.sum(params) - params[i - imin])
    return p_i

def posterior_pi_mean_95pdi(i, params):
    p = posterior_marginal_dist_pi(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

def posterior_nords_mean(params):
    ex = 0
    for i in range(1, len(params)+1):
        m = posterior_marginal_dist_pi(i, params).mean()
        ex += i * m
    return ex

Nmax = 30
s = 1.5
nsample = 1000

exact_dist = stats.zipfian(a=s, n=Nmax)
data = exact_dist.rvs(nsample)

pars = initial_params_dr(Nmax)
pars = posterior_params_dr(data, pars)

post_samp = posterior_nords_rvs(pars, 100000)

pi = [posterior_pi_mean_95pdi(i, pars) for i in range(1, Nmax+1)]

x = np.arange(1, Nmax+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pmf(x), name='Exact'))
fig.add_trace(go.Histogram(x=data, histnorm='probability', name='Samp', nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability', name='Post Samp', opacity=0.5, nbinsx=round(Nmax*2)))
fig.add_trace(go.Scatter(x=x, 
                         y=[p[0] for p in pi],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
                         name='Posterior Params',
                         mode='markers'))
fig.update_layout(title='Zipf Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  xaxis_range=[0, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

Средние

P(группа A > группа B)

Мало отличается от бутстрапа.

In [ ]:
unique, counts = np.unique(data, return_counts=True)
display(dict(zip(unique, counts / np.sum(counts))))

pi = []
for i in range(1, Nmax+1):
    pi.append(posterior_pi_mean_95pdi(i, pars))
pi

# Средний чек

Можно моделировать распределение чеков лог-нормальным распределением.  
Но если пользователь делает несколько заказов, заказы вряд ли будут независимы. 

Общее распределение чеков складывается из распределений чеков пользователей с разным числом заказов.

Лучше использовать иерархическую модель.

Можно построить общее распределение чеков.  
Вероятность чека $x$:

$$
P(x) = P(n=1) P_{n=1}(x) + P(n=2) P_{n=2}(x) + \dots + P(n=N) P_{n=N}(x)
\\
P(n=i) - \mbox{вероятность у пользователя i заказов}
\\
P_{n=2}(x) = Binom(s=1, N=2, p_{x, n=2}) + Binom(s=2, N=2, p_{x, n=2}) = 1 - Binom(s=0, N=2, p_{x, n=2}) = 1 - (1 - p_{x, n=2})^2
\\
P(x) = \sum_{i=1}^N P(n=i) (1 - (1 - p_{x, n=i})^i)
$$

Распределение будет сложной формы.  
Вряд ли получится подобрать аналитическую модель.  

Можно оценить среднее с помощью центральной предельной теоремы.  

Среднее (точечная оценка) должна совпасть с выручка/заказы.  
Интервал нужно собирать с учетом количества покупок пользователя.

# Сравнение распределений

In [ ]:
# params = {
#     'A': {'b': 2.8}
#     'B': {'b': 2.9}
# }

b = 1.3
scale = 1

exact_dist_a = stats.pareto(b=b)
exact_dist_b = stats.pareto(b=b*1.05)

nsample = 1000

samp_a = exact_dist_a.rvs(nsample)
samp_b = exact_dist_b.rvs(nsample)

alpha_prior = 2
beta_prior = 1

postdist_a = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_a)
postdist_b = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_b)
#postdist

x = np.linspace(0, 5, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_a.pdf(x), mode='lines', name=f'A'))
fig.add_trace(go.Scatter(x=x, y=postdist_b.pdf(x), mode='lines', name=f'B'))
fig.update_layout(title='Posterior Distribution',
                  xaxis_title='$alpha$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


post_nsamp = 50000
post_samp_a = postdist_a.rvs(alpha_nsamp)
post_samp_b = postdist_b.rvs(alpha_nsamp)
p_alpha_b_gt_alpha_a = np.sum(post_samp_b > post_samp_a) / post_nsamp
print(f'P(alpha_B > alpha_A) = {p_alpha_b_gt_alpha_a :.2f}')

# Ссылки

Fink, Daniel (1997). "A Compendium of Conjugate Priors"
https://courses.physics.ucsd.edu/2018/Fall/physics210b/REFERENCES/conjugate_priors.pdf  

Internet Mathematics Vol. 1, No. 2: 226-251  
A Brief History of Generative Models for Power Law and Lognormal Distributions  
Michael Mitzenmacher  
http://www.eecs.harvard.edu/~michaelm/postscripts/im2004a.pdf

[ConjPrior] - [Conjugate prior](https://en.wikipedia.org/wiki/Conjugate_prior), *Wikipedia.*   
[LognormDist] - [Log-normal Distribution](https://en.wikipedia.org/wiki/Log-normal_distribution), *Wikipedia.*   
[SciPyLognorm] - [scipy.stats.lognorm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html), *SciPy Reference.*   
[NormDist] - [Normal Distribution](https://en.wikipedia.org/wiki/Normal_distribution), *Wikipedia.*   
[SciPyNorm] - [scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html), *SciPy Reference.*   
[ParetoDist] - [Pareto Distribution](https://en.wikipedia.org/wiki/Pareto_distribution), *Wikipedia.*  
[SciPyPareto] - [scipy.stats.pareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html), *SciPy Reference.*   
[compendum of priors]  
[GammaDist] - [Gamma Distribution](https://en.wikipedia.org/wiki/Gamma_distribution), *Wikipedia.*   
[SciPyStatsGamma] - [scipy.stats.gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html), *SciPy Reference.* 

# Приложение

# Парето

Распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] 

$$
P(x; x_m, c) = 
\begin{cases}
\dfrac{c x_m^c}{x^{c + 1}} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for b, loc, scale in [(1, 0, 1), (2, 0, 1), (3, 0, 4), (4, 1, 1)]:
    fig.add_trace(go.Scatter(x=x, y=stats.pareto.pdf(x, b=b, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f'b={b}, loc={loc}, x_m={scale}'))
fig.update_layout(title='Pareto',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Сопряженное априорное распределение - Гамма-распределение
https://en.wikipedia.org/wiki/Conjugate_prior ,
[ссылка на a compendum of priors]   


$$
P(model) = Gamma(x; a, b) = \frac{b^a}{\Gamma(a)} x^{a-1} e^{-b x}, \quad x>0, \quad a,b>0 .
$$

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = 
\begin{cases}
\dfrac{\alpha}{x_m} \left(\dfrac{x}{x_m}\right)^{-(\alpha+1)} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{\alpha}{x_m}\left(\frac{x}{x_m}\right)^{-(\alpha + 1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(x/x_m)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(x/x_m))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(x/x_m))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(x_i/x_m))
$$

Среднее

$$
E[x] = 
\begin{cases}
\dfrac{\alpha x_m}{\alpha - 1} & \alpha > 1
\\
\infty & \alpha \le 1
\end{cases}
$$

Медиана

$$
m = x_m 2^{1/\alpha}
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for a,b in [(1,1), (2,1), (3,1), (9, 3), (45, 15), (90, 30)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Оценка параметров

In [ ]:
ConjugateParetoParams = namedtuple('ConjugateParetoParams', 'alpha beta x_m')

def initial_params_pareto(alpha=2, beta=1, x_m=1):
    return ConjugateParetoParams(alpha=alpha, beta=beta, x_m=x_m)

def posterior_params_pareto(data, initial_pars):
    alpha_post = initial_pars.alpha + len(data)
    beta_post = initial_pars.beta + np.sum([np.log(x/initial_pars.x_m) for x in data])
    return ConjugateParetoParams(alpha=alpha_post, beta=beta_post, x_m=initial_pars.x_m)

def posterior_pareto_rvs(params, nsamp):
    b = stats.gamma.rvs(a=params.alpha, scale=1/params.beta, size=nsamp)
    x = stats.pareto.rvs(b=b)
    return x

def posterior_shape_dist(params):
    b = stats.gamma(a=params.alpha, scale=1/params.beta)
    return b


b = 2.8
scale = 1
nsample = 1000

exact_dist = stats.pareto(b=b)
data = exact_dist.rvs(nsample)

pars = initial_params_pareto(alpha=2, beta=1, x_m=1)
pars = posterior_params_pareto(data, pars)

post_shape_dist = posterior_shape_dist(pars)
post_samp = posterior_pareto_rvs(pars, 50000)


x = np.linspace(-20, 20, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), name='Exact'))
fig.add_trace(go.Histogram(x=data, histnorm='probability density', name='Samp', nbinsx=300))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability density', name='Post Samp', opacity=0.5, nbinsx=10000))
fig.update_layout(title='Pareto Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()


x = np.linspace(0, 10, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_shape_dist.pdf(x), mode='lines', name=f'Posterior'))
fig.add_vline(b, name='b')
fig.update_layout(title='Pareto Shape Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

samp_b = post_shape_dist.rvs(size=50000)
samp_mean = samp_b * scale / (samp_b - 1)
samp_median = scale * 2**(1 / samp_b)

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_mean, histnorm='probability density', name='Mean', nbinsx=1000))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.update_layout(title='Mean',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_median, histnorm='probability density', name='Median', nbinsx=1000))
fig.add_vline(exact_dist.median(), name='Original Distribution Median')
fig.update_layout(title='Median',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.show()

## Сопряженное распределение к нормальному

Сопряженное априорное распределение для нормальной функции правдоподобия:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \mu, \sigma_x) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
$$

Один параметр: $\mu$ меняется, $\sigma$ фиксировано.  
Сопряженное априорное распределение - нормальное распределение.

$$
P(model) = Norm(\mu | \mu_0, \sigma_0) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
& \propto
e^{-\dfrac{(\mu - \mu_1)^2}{2 \sigma_1^2}}
= Norm(\mu | \mu_1, \sigma_1)
\end{split}
$$

$$
\dfrac{x^2 - 2 \mu x + \mu^2}{2 \sigma_x^2} + \dfrac{\mu^2 - 2\mu \mu_0 + \mu_0^2}{2 \sigma_{0}^2}
=
\mu^2 \left[ \frac{1}{2 \sigma_x^2} + \frac{1}{2 \sigma_0^2} \right] -
2 \mu \left[ \frac{x}{2 \sigma_x^2} + \frac{\mu_0}{2 \sigma_{0}^2} \right]  + C
=
\dfrac{\mu^2 - 2 \mu \mu_1 + \mu_1^2}{2 \sigma_1^2} - \frac{\mu_1^2}{2 \sigma_1^2} + C
$$

$$
\frac{1}{2 \sigma_1^2} = \left[ \frac{1}{2 \sigma_x^2} + \frac{1}{2 \sigma_0^2} \right],
\quad
\frac{\mu_1}{2 \sigma_1^2} = \left[ \frac{x}{2 \sigma_x^2} + \frac{\mu_0}{2 \sigma_{0}^2} \right]
\\
\sigma_1^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_0^2 + \sigma_x^2},
\quad
\mu_1 = \frac{x \sigma_0^2 + \mu_0 \sigma_x^2}{\sigma_0^2 + \sigma_x^2}
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x_i - \mu)^2}{2 \sigma_x^2}}
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
& \propto
e^{-\dfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N)
\end{split}
$$

$$
\dfrac{\mu^2 - 2\mu \mu_0 + \mu_0^2}{2 \sigma_{0}^2} + \sum_i^N \dfrac{x_i^2 - 2 \mu x_i + \mu^2}{2 \sigma_x^2}
=
\mu^2 \left[ \frac{1}{2 \sigma_0^2} + \frac{N}{2 \sigma_x^2} \right] -
2 \mu \left[ \frac{\mu_0}{2 \sigma_{0}^2} + \sum_i^N \frac{x_i}{2 \sigma_x^2} \right]  + C
=
\dfrac{\mu^2 - 2 \mu \mu_N + \mu_N^2}{2 \sigma_N^2} - \frac{\mu_N^2}{2 \sigma_N^2} + C
$$

$$
\frac{1}{2 \sigma_N^2} = \left[ \frac{1}{2 \sigma_0^2} + \frac{N}{2 \sigma_x^2} \right],
\quad
\frac{\mu_N}{2 \sigma_N^2} = \left[ \frac{\mu_0}{2 \sigma_{0}^2} + \sum_i^N \frac{x_i}{2 \sigma_x^2} \right]
\\
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \frac{\mu_0 \sigma_x^2 + \sigma_0^2 \sum_i^N x_i}{\sigma_x^2 + N \sigma_0^2 }
$$

Один параметр: $\sigma$ меняется, $\mu$ фиксировано.  
Сопряженное априорное распределение - гамма-распределение.

$$
P(model) = Gamma(\tau; a, b) = \frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{\tau^{1/2}}{\sqrt{\pi}} e^{-\tau (x - \mu_x)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
& \propto
\tau^{a-1+1/2} e^{-b \tau - \tau (x - \mu_x)^2 }
= Gamma(\tau; a+1/2, b + (x - \mu_x)^2)
\end{split}
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
\frac{\tau^{1/2}}{\sqrt{\pi}} e^{-\tau (x_i - \mu_x)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
& \propto
\tau^{a-1+N/2} e^{-b \tau - \tau \sum_i^N (x_i - \mu_x)^2 }
= Gamma(\tau; a+N/2, b + \sum_i^N (x_i - \mu_x)^2)
\end{split}
$$

Два параметра: $\mu, \sigma$.  

$$
P(data | model) = P(x | \mu, \sigma_x) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
$$

Сопряженное априорное распределение - произведение нормального и гамма-распределений.

$$
P(model) = 
Norm(\mu | \tau; \mu_0, k_0) Gamma(\tau; a, b) = 
(\tau k_0/ \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$


$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
(\tau/ \pi)^{1/2} e^{- \tau (x-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_1 (\mu-\mu_1)^2}
\tau^{a_1-1} e^{-b_1 \tau}
\\
& = Norm(\mu | \tau; \mu_1, k_1) Gamma(\tau; a_1, b_1)
\end{split}
$$

$$
a_1 = a + 1/2
$$

$$
(x-\mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + 1) - 2 \mu (x + k_0 \mu_0) + x^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + 1) \left[ \mu^2 - 2 \mu \frac{x + k_0 \mu_0}{k_0 + 1} + \left(\frac{x + k_0 \mu_0}{k_0 + 1}\right)^2 \right]
- \frac{(x + k_0 \mu_0)^2}{k_0 + 1} + x^2 + k_0 \mu_0^2 + b
\\
= k_1 (\mu-\mu_1)^2 + b_1
$$

$$
k_1 = k_0 + 1
\\
\mu_1 = \frac{x + k_0 \mu_0}{k_0 + 1}
\\
b_1 = b + x^2 + k_0 \mu_0^2 - \frac{(x + k_0 \mu_0)^2}{k_0 + 1}
\\
= b + k_0 \mu_0^2 - k_1 \mu_1^2 + x^2
$$

$$
b_1 = b + \frac{x^2 k_0 + x^2 + k_0^2 \mu_0^2 + k_0 \mu_0^2 - x^2 - 2xk_0\mu_0 - k_0^2 \mu_0^2}{k_0+1}
= b + \frac{x^2 k_0 + k_0 \mu_0^2 - 2xk_0\mu_0}{k_0+1}
\\
= b + \frac{k_0}{k_0+1}(x - \mu_0)^2
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
\prod_i^N (\tau/ \pi)^{1/2} e^{- \tau (x_i-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& = Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\end{split}
$$

$$
a_N = a + N/2
$$

$$
\sum_i^N (x_i - \mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + N) - 2 \mu (k_0 \mu_0 + \sum_i^N x_i) + \sum_i^N x_i^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + N) \left[ \mu^2 - 2 \mu \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N} + 
\left(\frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}\right)^2 \right]
- \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N} + \sum_i^N x_i^2 + k_0 \mu_0^2 + b
\\
= k_N (\mu-\mu_N)^2 + b_N
$$

$$
k_N = k_0 + N
\\
\mu_N = \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}
\\
b_N = b + \sum_i^N x_i^2 + k_0 \mu_0^2 - \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N}
\\
b_N = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N x_i^2
$$

Маржинальные распределения

По $\tau$ гамма-распределение

$$
P(\tau | data) = Gamma(\tau; a_N, b_N)
$$

По $\mu$ $t$-распределение  
https://en.wikipedia.org/wiki/Student%27s_t-distribution#Location-scale_t_distribution  

$$
\begin{split}
P(\mu | data) & = \int d\tau P(\mu, \tau | data)
\\
& = \int d\tau Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\\
& \propto_{\mu}
\int d\tau 
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& \propto_{\mu}
\int d\tau
\tau^{a_N + 1/2 -1} e^{-\tau (b_N + k_N (\mu-\mu_N)^2)}
\\
& \propto_{\mu}
\frac{\Gamma(a_N + 1/2)}{(b_N + k_N (\mu-\mu_N)^2)^{a_N + 1/2}}
\\
& \propto_{\mu}
\left(1 + \frac{1}{2 a_N} \frac{2 a_N k_N}{b_N} (\mu-\mu_N)^2\right)^{-(2 a_N + 1)/2}
\\
& = lst \left( \mu; \nu = 2 a_N, \mu_t = \mu_N, \tau= \left( \frac{b_N}{2 a_N k_N} \right)^{1/2} \right)
\end{split}
$$

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu k a b')

def initial_params_normal(mu, k=1, a=2, b=1):
    return ConjugateNormalParams(mu=mu, k=k, a=a, b=b)

def posterior_params_normal(data, initial_pars):
    N = len(data)
    a_n = initial_pars.a + N / 2
    k_n = initial_pars.k + N
    mu_n = (initial_pars.k * initial_pars.mu + np.sum(data)) / (initial_pars.k + N)    
    b_n = initial_pars.b + initial_pars.k * initial_pars.mu**2 - k_n * mu_n**2 + np.sum(data*data)
    return ConjugateNormalParams(mu=mu_n, k=k_n, a=a_n, b=b_n)

def posterior_normal_rvs(params, nsamp):
    tau = stats.gamma.rvs(a=params.a, scale=1/params.b, size=nsamp)
    sigma_m = 1 / np.sqrt(2 * params.k * tau)
    mu = stats.norm.rvs(loc=params.mu, scale=sigma_m, size=nsamp)
    sigma_x = 1 / np.sqrt(2 * tau)
    x = stats.norm.rvs(loc=mu, scale=sigma_x, size=nsamp)
    return x

def posterior_normal_marginal_mu_dist(params):
    nu = 2 * params.a
    mu_t = params.mu
    tau = np.sqrt(params.b/(2.0 * params.a * params.k))
    return stats.t(df=nu, loc=mu_t, scale=tau)

def posterior_normal_marginal_tau_dist(params):
    return stats.gamma(a=params.a, scale=1/params.b)


loc = 1
scale = 5
nsample = 300

exact_dist_norm = stats.norm(loc=loc, scale=scale)

data_norm = exact_dist_norm.rvs(nsample)

pars_norm = initial_params_normal(mu=1, k=1, a=2, b=1)
pars_norm = posterior_params_normal(data_norm, pars_norm)

post_samp_normal = posterior_normal_rvs(pars_norm, nsamp=200000)
postdist_normal_mu = posterior_normal_marginal_mu_dist(pars_norm)
postdist_normal_tau = posterior_normal_marginal_tau_dist(pars_norm)
#postdist

x = np.linspace(-20, 20, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist_norm.pdf(x), name='Norm'))
fig.add_trace(go.Histogram(x=post_samp_normal, histnorm='probability density', name='Post Samp Hist', nbinsx=1000))
fig.update_layout(title='Normal Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  #xaxis_range=[0,10],
                  hovermode="x",
                  height=550)
fig.show()


x = np.linspace(-1, 10, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_normal_mu.pdf(x), mode='lines', name=f'Posterior Mu'))
fig.add_vline(exact_dist_norm.mean(), name='Original Distribution Mean')
fig.update_layout(title='Mu Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

x = np.linspace(-3, 3, 20000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_normal_tau.pdf(x), mode='lines', name=f'Posterior Tau'))
fig.add_vline(1/(2 * exact_dist_norm.std()**2), name='Original Distribution Sigma')
fig.update_layout(title='Tau Distribution',
                  xaxis_title='$x$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

## Лог-нормальное

$$
P(data | model) = P(x | \mu, \sigma) = 
\frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
$$

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
(\tau/ x \pi)^{1/2} e^{- \tau (\ln(x)-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_1 (\mu-\mu_1)^2}
\tau^{a_1-1} e^{-b_1 \tau}
\\
& = Norm(\mu | \tau; \mu_1, k_1) Gamma(\tau; a_1, b_1)
\end{split}
$$

$$
a_1 = a + 1/2
\\
k_1 = k_0 + 1
\\
\mu_1 = \frac{k_0 \mu_0 + \ln(x)}{k_0 + 1}
\\
b_1 = b + k_0 \mu_0^2 - k_1 \mu_1^2 + (\ln x)^2
$$

## Ломакс

Плотность распределения

$$
p(x) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}, \qquad x \geq 0
$$

$$
p(x) = {{\alpha\lambda^\alpha} \over {(x + \lambda)^{\alpha+1}}}
$$

Для распределения на пользователей: обезразмерить на $x_m$.  
Потом Ломакс с $\lambda=1$.  
Проще сразу безразмерный вариант распределения.

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}
= \frac{\alpha}{\lambda}\frac{1}{\left[{1 + (x/\lambda)}\right]^{(\alpha+1)}}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
{\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(1 + x/\lambda)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(1 + x/\lambda))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(1 + x/\lambda))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(1 + x_i/\lambda))
$$

## Сопряженное распределение для разности CDF Парето

Гамма-распределение:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = \dfrac{1}{n^s} - \dfrac{1}{(n+1)^{s}}
$$

$$
P(model) = Gamma(s; a, b) = \frac{b^a}{\Gamma(a)} s^{a-1} e^{-b s}, \quad s>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\left( \dfrac{1}{n^s} - \dfrac{1}{(n+1)^{s}} \right) \frac{b^a}{\Gamma(a)} s^{a-1} e^{-b s}
\\
& \propto
\left( e^{-s \ln n} - e^{-s \ln (n+1)} \right) \frac{b^a}{\Gamma(a)} s^{a-1} e^{-b s}
\\
& \propto
Gamma(s; a, b + \ln(n)) - Gamma(s; a, b + \ln(n+1))
\end{split}
$$


Экспоненциальное:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = \dfrac{1}{n^s} - \dfrac{1}{(n+1)^{s}}
$$

$$
P(model) = \begin{cases}
\lambda e^{-\lambda (s - 1)} & s \geq 1,
\\
0 & s < 1.
\end{cases}
$$

$$
\begin{split}
P(model | data) 
& \propto
\left( \dfrac{1}{n^s} - \dfrac{1}{(n+1)^{s}} \right) \lambda e^{-\lambda (s - 1)}
\\
& \propto
\left( e^{-s \ln n} - e^{-s \ln (n+1)} \right) \lambda e^{-\lambda (s - 1)}
\\
& \propto
\lambda e^{-\lambda (s - 1) - s \ln n} - \lambda e^{-\lambda (s - 1) - s \ln (n+1)}
\end{split}
$$

## Сопряженное распределение для распределение Ципфа 

Подбирается сложно из-за зависящих от $s$ нормировочных коэффициентов

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = \frac{1}{H_{N,s}} \frac{1}{k^s}
$$

$$
P(model) = Gamma(s; a, b) = \frac{b^a}{\Gamma(a)} s^{a-1} e^{-b s}, \quad s>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{1}{\sum_{n=1}^N n^{-s}} k^{-s} \frac{b^a}{\Gamma(a)} s^{a-1} e^{-b s}
\end{split}
$$

**Ожидание произведения**

Можно поделить выручку на пользователя на заказы на пользователя. 
Количество заказов и выручка скоррелированы.  
Если просто поделить - корреляция не будет учтена.  
Нужна аккуратность с зависимыми величинами

$$
E[XY] \ne E[X] E[Y]
\\
E[X Y] = E[X] E[Y] + cov(X,Y)
$$